In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datasets import Dataset, DatasetDict

from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

import torch
#from sentence_transformers import SentenceTransformer, InputExample, losses, models, evaluation
from torch.utils.data import DataLoader
#from sentence_transformers import LoggingHandler
#from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import logging
import numpy as np
import re

#from sklearn.metrics import roc_curve, auc

from tqdm.auto import tqdm
from tqdm.notebook import tqdm
from transformers import AutoModel, AutoTokenizer


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_folder = "C:\\Users\\gckc1\\My Drive\\My programs\\Sierra\\full_auto_review_data\\20240115_for_model_training"
model_folder = "C:\\Users\\gckc1\\My Drive\\My programs\\Sierra\\model_training"
review_data_folder = "C:\\Users\\gckc1\\My Drive\\My programs\\Sierra\\review_list"


In [2]:
val_1_file_location = "20240419_val_1_rti_bg_obj_sel_output.csv"
val_1_file = pd.read_csv(data_folder + "\\" + val_1_file_location)

val_2_file_location = "20240419_val_2_rti_bg_obj_sel_output.csv"
val_2_file = pd.read_csv(data_folder + "\\" + val_2_file_location)

dev_file_location = "20240417_dev_set_shuffled.csv"
dev_file = pd.read_csv(data_folder + "\\" + dev_file_location)

val_heart_file_location = "20240419_val_heart_rti_bg_obj_sel_output.csv"
val_heart_file = pd.read_csv(data_folder + "\\" + val_heart_file_location) 

val_HIV_file_location = "20240419_val_HIV_rti_bg_obj_sel_output.csv"
val_HIV_file = pd.read_csv(data_folder + "\\" + val_HIV_file_location) 


In [3]:
val_HIV_file = val_HIV_file.drop(['obj_sel_input_ids', 'token_type_ids', "obj_sel_attention_mask", "tit_abs_input_ids","tit_abs_attention_mask"], axis=1).copy()
val_heart_file = val_heart_file.drop(['obj_sel_input_ids', 'token_type_ids', "obj_sel_attention_mask", "tit_abs_input_ids","tit_abs_attention_mask"], axis=1).copy()
val_1_file = val_1_file.drop(['obj_sel_input_ids', 'token_type_ids', "obj_sel_attention_mask", "tit_abs_input_ids","tit_abs_attention_mask"], axis=1).copy()
val_2_file = val_2_file.drop(['obj_sel_input_ids', 'token_type_ids', "obj_sel_attention_mask", "tit_abs_input_ids","tit_abs_attention_mask"], axis=1).copy()

In [4]:
recode_dict = {0:0, 0.5:1, 1:2}
dev_file["label"] = dev_file["label"].replace(recode_dict)




In [5]:
dev_file['label'].value_counts()

label
0.0    224711
1.0    161074
2.0     81190
Name: count, dtype: int64

In [6]:
dev_file["obj_sel"] = dev_file["obj_sel"].str.replace(r'^\[OA\]', '', regex=True)
dev_file["tit_abs"] = dev_file["tit_abs"].str.replace(r'^ \[OA\]', '', regex=True)

In [7]:
dev_file["obj_sel"] = "[RTI]" + dev_file["Review_Title"] + dev_file["obj_sel"]

In [8]:
dev_file_1 = dev_file[0:100000].copy()
dev_file_2 = dev_file[100000:200000].copy()
dev_file_3 = dev_file[200000:300000].copy()
dev_file_4 = dev_file[300000:466975].copy()


In [9]:
val_HIV_set = Dataset.from_pandas(val_HIV_file)
val_1_set = Dataset.from_pandas(val_1_file)
val_2_set = Dataset.from_pandas(val_2_file)
dev_set_1 = Dataset.from_pandas(dev_file_1)
dev_set_2 = Dataset.from_pandas(dev_file_2)
dev_set_3 = Dataset.from_pandas(dev_file_3)
dev_set_4 = Dataset.from_pandas(dev_file_4)
val_heart_set = Dataset.from_pandas(val_heart_file)


In [10]:
model_ckpt = "dmis-lab/biobert-large-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [11]:
new_tokens = ["[RIT]", "[OBJ]", "[BG]", "[SEL]", "[TIT]", "[ABS]"]
num_added_toks = tokenizer.add_tokens(new_tokens)

In [12]:
# Check if the new tokens are in the tokenizer
for token in new_tokens:
    if token in tokenizer.get_vocab():
        print(f"Token {token} is in the vocabulary.")
    else:
        print(f"Token {token} is not in the vocabulary.")

# Check the size of the new embeddings
print(f"New embedding size: {model.get_input_embeddings().num_embeddings}")

model.resize_token_embeddings(len(tokenizer))

Token [RIT] is in the vocabulary.
Token [OBJ] is in the vocabulary.
Token [BG] is in the vocabulary.
Token [SEL] is in the vocabulary.
Token [TIT] is in the vocabulary.
Token [ABS] is in the vocabulary.
New embedding size: 58996


Embedding(59002, 1024)

In [13]:
for part in ["obj_sel", "tit_abs"]:
    val_HIV_set = val_HIV_set.map(
        lambda x:  tokenizer(
            x[part], max_length = 512, padding = True, truncation = True
        ), batched = True, batch_size = None
    )
    for col in ['input_ids', 'attention_mask']:
        val_HIV_set = val_HIV_set.rename_column(
            col, part+"_"+col
        )


for part in ["obj_sel", "tit_abs"]:
    dev_set_1 = dev_set_1.map(
        lambda x:  tokenizer(
            x[part], max_length = 512, padding = True, truncation = True
        ), batched = True, batch_size = None
    )
    for col in ['input_ids', 'attention_mask']:
        dev_set_1 = dev_set_1.rename_column(
            col, part+"_"+col
        )

for part in ["obj_sel", "tit_abs"]:
    dev_set_2 = dev_set_2.map(
        lambda x:  tokenizer(
            x[part], max_length = 512, padding = True, truncation = True
        ), batched = True, batch_size = None
    )
    for col in ['input_ids', 'attention_mask']:
        dev_set_2 = dev_set_2.rename_column(
            col, part+"_"+col
        )

for part in ["obj_sel", "tit_abs"]:
    dev_set_3 = dev_set_3.map(
        lambda x:  tokenizer(
            x[part], max_length = 512, padding = True, truncation = True
        ), batched = True, batch_size = None
    )
    for col in ['input_ids', 'attention_mask']:
        dev_set_3 = dev_set_3.rename_column(
            col, part+"_"+col
        )

for part in ["obj_sel", "tit_abs"]:
    dev_set_4 = dev_set_4.map(
        lambda x:  tokenizer(
            x[part], max_length = 512, padding = True, truncation = True
        ), batched = True, batch_size = None
    )
    for col in ['input_ids', 'attention_mask']:
        dev_set_4 = dev_set_4.rename_column(
            col, part+"_"+col
        )

for part in ["obj_sel", "tit_abs"]:
    val_1_set = val_1_set.map(
        lambda x:  tokenizer(
            x[part], max_length = 512, padding = True, truncation = True
        ), batched = True, batch_size = None
    )
    for col in ['input_ids', 'attention_mask']:
        val_1_set = val_1_set.rename_column(
            col, part+"_"+col
        )

for part in ["obj_sel", "tit_abs"]:
    val_2_set = val_2_set.map(
        lambda x:  tokenizer(
            x[part], max_length = 512, padding = True, truncation = True
        ), batched = True, batch_size = None
    )
    for col in ['input_ids', 'attention_mask']:
        val_2_set = val_2_set.rename_column(
            col, part+"_"+col
        )

for part in ["obj_sel", "tit_abs"]:
    val_heart_set = val_heart_set.map(
        lambda x:  tokenizer(
            x[part], max_length = 512, padding = True, truncation = True
        ), batched = True, batch_size = None
    )
    for col in ['input_ids', 'attention_mask']:
        val_heart_set = val_heart_set.rename_column(
            col, part+"_"+col
        )

all_cols = ['label', 'obj_sel_input_ids', 'obj_sel_attention_mask', 'tit_abs_input_ids', 'tit_abs_attention_mask']


Map:   0%|          | 0/4400 [00:00<?, ? examples/s]

Map:   0%|          | 0/4400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/26892 [00:00<?, ? examples/s]

Map:   0%|          | 0/27051 [00:00<?, ? examples/s]

Map:   0%|          | 0/27051 [00:00<?, ? examples/s]

Map:   0%|          | 0/18839 [00:00<?, ? examples/s]

Map:   0%|          | 0/18839 [00:00<?, ? examples/s]

In [14]:
tmp1 = dev_set_1.to_pandas()
tmp2 = dev_set_2.to_pandas()
tmp3 = dev_set_3.to_pandas()
tmp4 = dev_set_4.to_pandas()

tmp = pd.concat([tmp1, tmp2, tmp3, tmp4], ignore_index = True)

dev_set = Dataset.from_pandas(tmp)

In [15]:
dev_set.set_format(type = 'torch', columns = all_cols)

In [16]:
batch_size = 32
loader = torch.utils.data.DataLoader(
    dev_set, batch_size=batch_size, shuffle = True)

In [17]:
def mean_pool(token_embeds, attention_mask):
    in_mask = attention_mask.unsqueeze(-1).expand(token_embeds.size()).float()
    pool = torch.sum(token_embeds * in_mask, 1) / torch.clamp(
        in_mask.sum(1), min=1e-9
    )
    return pool
    

In [18]:
ffnn = torch.nn.Linear(1024*3, 3)
loss_func = torch.nn.CrossEntropyLoss()

In [19]:
from transformers.optimization import get_linear_schedule_with_warmup

optim = torch.optim.Adam(model.parameters(), lr = 2e-5)
total_steps = int(len(dev_set) / batch_size)
warmup_steps = 500
scheduler = get_linear_schedule_with_warmup(
    optim, num_warmup_steps=warmup_steps,
    num_training_steps=total_steps - warmup_steps
)




In [20]:
import torch.nn as nn

model = nn.DataParallel(model)
ffnn = nn.DataParallel(ffnn)

In [21]:
model = model.to(device)
ffnn = ffnn.to(device)

In [22]:
for epoch in range(1):
    model.train()
    loop = tqdm(loader, leave= True)
    for batch in loop:
        optim.zero_grad()
        inputs_ids_a = batch['obj_sel_input_ids'].to(device)
        inputs_ids_b = batch['tit_abs_input_ids'].to(device)
        attention_a = batch['obj_sel_attention_mask'].to(device)
        attention_b = batch['tit_abs_attention_mask'].to(device)
        label = batch['label'].to(device).long()
        u = model(inputs_ids_a, attention_mask = attention_a)[0]
        v = model(inputs_ids_b, attention_mask = attention_b)[0]
        u = mean_pool(u, attention_a)
        v = mean_pool(v, attention_b)
        uv = torch.sub(u, v)
        uv_abs = torch.abs(uv)
        x = torch.cat([u, v, uv_abs], dim = -1)
        x = ffnn(x)
        loss = loss_func(x, label)
        loss.backward()
        optim.step()
        scheduler.step()
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/14593 [00:00<?, ?it/s]

C:\Users\gckc1\anaconda3\envs\Sierra\Lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn("PyTorch is not compiled with NCCL support")


In [23]:
torch.save(model, model_folder + "\\20240420_biobert_logit_title_bg\\" + '20240420_biobert_logit_466975.pth')
torch.save(ffnn, model_folder + "\\20240420_biobert_logit_title_bg\\" + '20240420_ffnn_logit_466975.pth')

In [ ]:
val_1_file_location = "20240419_val_1_rti_bg_obj_sel_output.csv"
val_1_file = pd.read_csv(data_folder + "\\" + val_1_file_location)

val_2_file_location = "20240419_val_2_rti_bg_obj_sel_output.csv"
val_2_file = pd.read_csv(data_folder + "\\" + val_2_file_location)

val_heart_file_location = "20240419_val_heart_rti_bg_obj_sel_output.csv"
val_heart_file = pd.read_csv(data_folder + "\\" + val_heart_file_location) 

val_HIV_file_location = "20240419_val_HIV_rti_bg_obj_sel_output.csv"
val_HIV_file = pd.read_csv(data_folder + "\\" + val_HIV_file_location) 

val_HIV_file = val_HIV_file.drop(['obj_sel_input_ids', 'token_type_ids', "obj_sel_attention_mask", "tit_abs_input_ids","tit_abs_attention_mask"], axis=1).copy()
val_heart_file = val_heart_file.drop(['obj_sel_input_ids', 'token_type_ids', "obj_sel_attention_mask", "tit_abs_input_ids","tit_abs_attention_mask"], axis=1).copy()
val_1_file = val_1_file.drop(['obj_sel_input_ids', 'token_type_ids', "obj_sel_attention_mask", "tit_abs_input_ids","tit_abs_attention_mask"], axis=1).copy()
val_2_file = val_2_file.drop(['obj_sel_input_ids', 'token_type_ids', "obj_sel_attention_mask", "tit_abs_input_ids","tit_abs_attention_mask"], axis=1).copy()

val_1_set = Dataset.from_pandas(val_1_file)
val_2_set = Dataset.from_pandas(val_2_file)
val_heart_set = Dataset.from_pandas(val_heart_file)
val_HIV_set = Dataset.from_pandas(val_HIV_file)

for part in ["obj_sel", "tit_abs"]:
    val_1_set = val_1_set.map(
        lambda x:  tokenizer(
            x[part], max_length = 512, padding = True, truncation = True
        ), batched = True, batch_size = None
    )
    for col in ['input_ids', 'attention_mask']:
        val_1_set = val_1_set.rename_column(
            col, part+"_"+col
        )

for part in ["obj_sel", "tit_abs"]:
    val_2_set = val_2_set.map(
        lambda x:  tokenizer(
            x[part], max_length = 512, padding = True, truncation = True
        ), batched = True, batch_size = None
    )
    for col in ['input_ids', 'attention_mask']:
        val_2_set = val_2_set.rename_column(
            col, part+"_"+col
        )

for part in ["obj_sel", "tit_abs"]:
    val_heart_set = val_heart_set.map(
        lambda x:  tokenizer(
            x[part], max_length = 512, padding = True, truncation = True
        ), batched = True, batch_size = None
    )
    for col in ['input_ids', 'attention_mask']:
        val_heart_set = val_heart_set.rename_column(
            col, part+"_"+col
        )

for part in ["obj_sel", "tit_abs"]:
    val_HIV_set = val_HIV_set.map(
        lambda x:  tokenizer(
            x[part], max_length = 512, padding = True, truncation = True
        ), batched = True, batch_size = None
    )
    for col in ['input_ids', 'attention_mask']:
        val_HIV_set = val_HIV_set.rename_column(
            col, part+"_"+col
        )



In [24]:
val_1_set.set_format(type = 'torch', columns = all_cols)
val_2_set.set_format(type = 'torch', columns = all_cols)
val_heart_set.set_format(type = 'torch', columns = all_cols)
val_HIV_set.set_format(type = 'torch', columns = all_cols)

In [25]:
from torch.utils.data import DataLoader
val_1_loader = DataLoader(val_1_set, batch_size = 32, shuffle = False)
val_2_loader = DataLoader(val_2_set, batch_size = 32, shuffle = False)
val_heart_loader = DataLoader(val_heart_set, batch_size = 32, shuffle = False)
val_HIV_loader = DataLoader(val_HIV_set, batch_size = 32, shuffle = False)

In [26]:
def evaluate_model(model, ffnn, data_loader, device):
    model.eval()
    ffnn.eval()
    all_outputs = []
    predictions = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating", leave = True):
            inputs_ids_a = batch['obj_sel_input_ids'].to(device)
            inputs_ids_b = batch['tit_abs_input_ids'].to(device)
            attention_a = batch['obj_sel_attention_mask'].to(device)
            attention_b = batch['tit_abs_attention_mask'].to(device)
            labels = batch['label'].to(device).long()

            u = model(inputs_ids_a, attention_mask = attention_a)[0]
            v = model(inputs_ids_b, attention_mask = attention_b)[0]
            u = mean_pool(u, attention_a)
            v = mean_pool(v, attention_b)
            uv = torch.sub(u, v)
            uv_abs = torch.abs(uv)
            outputs = torch.cat([u, v, uv_abs], dim= -1)
            outputs = ffnn(outputs)

            _, predicted = torch.max(outputs, 1)

            predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_outputs.append(outputs.cpu().numpy())
    all_outputs = np.vstack(all_outputs)
    return predictions, all_labels, all_outputs

In [27]:
val_1_predictions, val_1_labels, val_1_outputs = evaluate_model(model, ffnn, val_1_loader, device)
val_2_predictions, val_2_labels, val_2_outputs = evaluate_model(model, ffnn, val_2_loader, device)
val_heart_predictions, val_heart_labels, val_heart_outputs = evaluate_model(model, ffnn, val_heart_loader, device)
val_HIV_predictions, val_HIV_labels, val_HIV_outputs = evaluate_model(model, ffnn, val_HIV_loader, device)


Evaluating:   0%|          | 0/841 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/846 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/589 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/138 [00:00<?, ?it/s]

In [28]:
def evaluate_prediction(df, predictions):
    df_tmp = df.copy()
    ct = pd.crosstab(df_tmp["Actual Labels"], df_tmp[predictions])
    print(ct)
    row_percentage = ct.div(ct.sum(axis = 1), axis = 0)
    print(row_percentage)

In [ ]:
################# Evaluation 20240421 #########################

In [34]:
val_1_pd = val_1_set.to_pandas()
val_2_pd = val_2_set.to_pandas()
val_heart_set_pd = val_heart_set.to_pandas()
val_HIV_set_pd = val_HIV_set.to_pandas()

In [31]:
def combine_outputs(df, labels, predictions):
    predictions_df = pd.DataFrame({
        'Predictions': predictions,
    })  
    combined = pd.concat([df, predictions_df], axis = 1)
    return combined.copy()

In [35]:
val_1_pd = combine_outputs(val_1_pd, val_1_labels, val_1_predictions)
val_2_pd = combine_outputs(val_2_pd, val_2_labels, val_2_predictions)
val_heart_set_pd = combine_outputs(val_heart_set_pd, val_heart_labels, val_heart_predictions)
val_HIV_set_pd = combine_outputs(val_HIV_set_pd, val_HIV_labels, val_HIV_predictions)

In [36]:
#Only need to run it once#

val_1_pd['DOI'] = val_1_pd['Review_URL'].str.extract(r'(10\.\d{4,9}/[-._;()/:A-Z0-9]+)', flags=re.IGNORECASE)
val_1_pd['DOI'] = val_1_pd['DOI'].str.replace(r'/full$', '', regex=True)

val_2_pd['DOI'] = val_2_pd['Review_URL'].str.extract(r'(10\.\d{4,9}/[-._;()/:A-Z0-9]+)', flags=re.IGNORECASE)
val_2_pd['DOI'] = val_2_pd['DOI'].str.replace(r'/full$', '', regex=True)

#Only need to run it once#

val_heart_set_pd['DOI'] = val_heart_set_pd['Review_URL'].str.extract(r'(10\.\d{4,9}/[-._;()/:A-Z0-9]+)', flags=re.IGNORECASE)
val_heart_set_pd['DOI'] = val_heart_set_pd['DOI'].str.replace(r'/full$', '', regex=True)

val_HIV_set_pd['DOI'] = val_HIV_set_pd['Review_URL'].str.extract(r'(10\.\d{4,9}/[-._;()/:A-Z0-9]+)', flags=re.IGNORECASE)
val_HIV_set_pd['DOI'] = val_HIV_set_pd['DOI'].str.replace(r'/full$', '', regex=True)

In [37]:
review_record = pd.read_csv(review_data_folder + "\\" + "20231218_cochrane_review_expertise.csv")

In [38]:
val_1_pd["expertise"] = val_1_pd['DOI'].isin(review_record['DOI'])
val_2_pd["expertise"] = val_2_pd['DOI'].isin(review_record['DOI'])
val_heart_set_pd["expertise"] = val_heart_set_pd['DOI'].isin(review_record['DOI'])
val_HIV_set_pd["expertise"] = val_HIV_set_pd['DOI'].isin(review_record['DOI'])

In [39]:
val_1_pd.to_csv(data_folder + "\\" + "20240420_val_1_rti_bg_obj_sel_output_logit.csv")
val_2_pd.to_csv(data_folder + "\\" + "20240420_val_2_rti_bg_obj_sel_output_logit.csv")
val_heart_set_pd.to_csv(data_folder + "\\" + "20240420_val_heart_rti_bg_obj_sel_output_logit.csv")
val_HIV_set_pd.to_csv(data_folder + "\\" + "20240420_val_HIV_rti_bg_obj_sel_output_logit.csv")

In [41]:
def evaluate_prediction(df, predictions):
    df_tmp = df.copy()
    ct = pd.crosstab(df_tmp["Actual Labels"], df_tmp[predictions])
    print(ct)
    row_percentage = ct.div(ct.sum(axis = 1), axis = 0)
    print(row_percentage)

In [42]:
print("*************************************************")
print("Val 1")
print("*************************************************")
evaluate_prediction(val_1_pd, "Predictions")
print("*************************************************")
print("Val 2")
print("*************************************************")
evaluate_prediction(val_2_pd, "Predictions")
print("*************************************************")
print("Heart")
print("*************************************************")
evaluate_prediction(val_heart_set_pd, "Predictions")
print("*************************************************")
print("HIV")
print("*************************************************")
evaluate_prediction(val_HIV_set_pd, "Predictions")

*************************************************
Val 1
*************************************************
Predictions        0     1     2
Actual Labels                   
0.0            11410   788   227
0.5             1427  7178  1317
1.0              129  2243  2173
Predictions           0         1         2
Actual Labels                              
0.0            0.918310  0.063421  0.018270
0.5            0.143822  0.723443  0.132735
1.0            0.028383  0.493509  0.478108
*************************************************
Val 2
*************************************************
Predictions        0     1     2
Actual Labels                   
0.0            11726  1068   226
0.5             1279  6668  1501
1.0               97  2132  2354
Predictions           0         1         2
Actual Labels                              
0.0            0.900614  0.082028  0.017358
0.5            0.135373  0.705758  0.158870
1.0            0.021165  0.465197  0.513637
******************

In [ ]:
def calculate_norm(df_relevance, output_column):
    df_tmp = df_relevance.copy()
    groups = [df1 for _, df1 in df_tmp.groupby("Objective")]
    for i in range(0, len(groups)):
        maximum = groups[i][output_column].max()
        minimum = groups[i][output_column].min()
        groups[i]["Normed relevance"] = (groups[i][output_column] - minimum)/(maximum - minimum)
        p95 = groups[i][output_column].quantile(0.95)
        p90 = groups[i][output_column].quantile(0.90)
        groups[i]["Normed relevance 95"] = (groups[i][output_column] - minimum)/(p95 - minimum)
        groups[i].loc[groups[i]["Normed relevance 95"] > 1, 'Normed relevance 95'] = 1 
        groups[i]["Normed relevance 90"] = (groups[i][output_column] - minimum)/(p90 - minimum)
        groups[i].loc[groups[i]["Normed relevance 90"] > 1, 'Normed relevance 90'] = 1
    df_norm = pd.concat(groups, ignore_index = True)
    return df_norm.copy()
    

In [ ]:
val_1_pd_n9095 = calculate_norm(val_1_pd, "Outputs")
val_2_pd_n9095 = calculate_norm(val_2_pd, "Outputs")
val_heart_pd_n9095 = calculate_norm(val_heart_set_pd, "Outputs")
val_HIV_pd_n9095 = calculate_norm(val_HIV_set_pd, "Outputs")

In [ ]:
len(val_heart_set_pd)

In [ ]:
print("******************************************")
print("Val 1")
print("******************************************")
roc_analysis(val_1_pd_n9095, "Normed relevance", save = True)
print("******************************************")
print("Val 2")
print("******************************************")
roc_analysis(val_2_pd_n9095, "Normed relevance")
print("******************************************")
print("Val Heart")
print("******************************************")
roc_analysis(val_heart_pd_n9095, "Normed relevance")
print("******************************************")
print("Val HIV")
print("******************************************")
roc_analysis(val_HIV_pd_n9095, "Normed relevance")

In [ ]:
evaluate_cutoff(val_1_pd_n9095, "Normed relevance", 0.566)

In [ ]:
evaluate_cutoff(val_2_pd_n9095, "Normed relevance", 0.566)

In [ ]:
evaluate_cutoff(val_heart_pd_n9095, "Normed relevance", 0.566)

In [ ]:
evaluate_cutoff(val_HIV_pd_n9095, "Normed relevance", 0.566)